## Import libraries

In [ ]:
import os
import gc
import glob
import joblib
import numpy.matlib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from joblib import Parallel, delayed
from IPython.core.display import display
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

import lightgbm as lgb
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.backend import sigmoid
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

np.random.seed(42)
tf.random.set_seed(42)

## Helper Functions

In [ ]:
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap


def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


def calc_depth(df):
    depth = df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1'] + df['bid_price2'] * df[
               'bid_size2'] + df['ask_price2'] * df['ask_size2']
    return depth


def calc_dispersion(df):
    bspread = df['bid_price1'] - df['bid_price2']
    aspread = df['ask_price2'] - df['ask_price1']
    bmid = (df['bid_price1'] + df['ask_price1'])/2  - df['bid_price1']
    bmid2 = (df['bid_price1'] + df['ask_price1'])/2  - df['bid_price2']
    amid = df['ask_price1'] - (df['bid_price1'] + df['ask_price1'])/2
    amid2 = df['ask_price2'] - (df['bid_price1'] + df['ask_price1'])/2
    bdisp = (df['bid_size1']*bmid + df['bid_size2']*bspread)/(df['bid_size1']+df['bid_size2'])
    bdisp2 = (df['bid_size1']*bmid + df['bid_size2']*bmid2)/(df['bid_size1']+df['bid_size2'])
    adisp = (df['ask_size1']*amid + df['ask_size2']*aspread)/(df['ask_size1']+df['ask_size2'])      
    adisp2 = (df['ask_size1']*amid + df['ask_size2']*amid2)/(df['ask_size1']+df['ask_size2'])
    return (bdisp + adisp)/2


def calc_slope(df):
    v0 = (df['bid_size1']+df['ask_size1'])/2
    p0 = (df['bid_price1']+df['ask_price1'])/2
    slope_bid = ((df['bid_size1']/v0)-1)/abs((df['bid_price1']/p0)-1)+(
                (df['bid_size2']/df['bid_size1'])-1)/abs((df['bid_price2']/df['bid_price1'])-1)
    slope_ask = ((df['ask_size1']/v0)-1)/abs((df['ask_price1']/p0)-1)+(
                (df['ask_size2']/df['ask_size1'])-1)/abs((df['ask_price2']/df['ask_price1'])-1)
    return (slope_bid+slope_ask)/2


def log_return(series):
    return np.log(series).diff()


def realized_volatility(series):
    return np.sqrt(np.sum(series**2))


def historical_volatility(series):
    return np.sqrt(np.sum(series**2)/len(series))


def count_unique(series):
    return len(np.unique(series))

    
def tendency(price, vol):    
    df_diff = np.diff(price)
    val = (df_diff/price[1:])*100
    power = np.sum(val*vol[1:])
    return(power)
    
    
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f"train: {train.shape} \ntest: {test.shape}")
    return train, test


def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))


def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False


def root_mean_squared_per_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))

In [ ]:
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)

    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    
    df['depth'] = calc_depth(df)
    df['slope'] = calc_slope(df)
    df['dispersion'] = calc_dispersion(df)
    
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    
    wap1_mean = df['wap1'].mean()
    df['waph1'] = df['wap1'].apply(lambda x: x - wap1_mean)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility, historical_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'waph1': [historical_volatility], 
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum, np.max],
        'depth':[np.sum, np.max],
        'slope':[np.sum, np.max],
        'dispersion':[np.sum, np.max]
    }
    
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'waph1': [historical_volatility]
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount'] = df['price'] * df['size']
    
    price_mean = df['price'].mean()
    df['price1'] = df['price'].apply(lambda x: x - price_mean)

    create_feature_dict = {
        'log_return':[realized_volatility, historical_volatility],
        'price1': [historical_volatility], 
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'price1': [historical_volatility], 
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({
            'time_id':n_time_id,
            'tendency':tendencyV,
            'f_max':f_max,
            'f_min':f_min,
            'df_max':df_max,
            'df_min':df_min,
            'abs_diff':abs_diff,
            'energy':energy,
            'iqr_p':iqr_p,
            'abs_diff_v':abs_diff_v,
            'energy_v':energy_v,
            'iqr_p_v':iqr_p_v
        })
    
    df_lr = pd.DataFrame(lis)
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

In [ ]:
def get_time_stock(df):
    vol_cols = [
        'log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_500', 'log_return2_realized_volatility_500', 
        'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 
        'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_500', 
        'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200'
    ]

    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    

def preprocessor(list_stock_ids, is_train = True):
    data_dir = '../input/optiver-realized-volatility-prediction/'
    
    def for_joblib(stock_id):
        
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        return df_tmp
    
    
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    df = pd.concat(df, ignore_index = True)
    return df

## Prepare data for model training

In [ ]:
train, test = read_train_test()

In [ ]:
# Process train data
train_stock_ids = train['stock_id'].unique()
train_ = preprocessor(train_stock_ids, is_train = True)
train = train.merge(train_, on = ['row_id'], how = 'left')
print(f"train: {train.shape}")

In [ ]:
# Process test data
test_stock_ids = test['stock_id'].unique()
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')
print(f"test: {test.shape}")

In [ ]:
# Create group features
train = get_time_stock(train)
test = get_time_stock(test)
print(f"train: {train.shape} \ntest: {test.shape}")

In [ ]:
# Tau features
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

print(f"train: {train.shape} \ntest: {test.shape}")

In [ ]:
colNames = [col for col in list(train.columns) if col not in {"stock_id", "time_id", "target", "row_id"}]
len(colNames)

In [ ]:
# Generate aggregate features
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()
ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )

mat = []
matTest = []

n = 0
for ind in l:
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)
mat2 = pd.concat(matTest).reset_index()

mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
nnn = [
    'time_id',
    'log_return1_realized_volatility_0c1',
    'log_return1_realized_volatility_1c1',     
    'log_return1_realized_volatility_3c1',
    'log_return1_realized_volatility_4c1',     
    'log_return1_realized_volatility_6c1',
    'total_volume_sum_0c1',
    'total_volume_sum_1c1', 
    'total_volume_sum_3c1',
    'total_volume_sum_4c1', 
    'total_volume_sum_6c1',
    'trade_size_sum_0c1',
    'trade_size_sum_1c1', 
    'trade_size_sum_3c1',
    'trade_size_sum_4c1', 
    'trade_size_sum_6c1',
    'trade_order_count_sum_0c1',
    'trade_order_count_sum_1c1',
    'trade_order_count_sum_3c1',
    'trade_order_count_sum_4c1',
    'trade_order_count_sum_6c1',      
    'price_spread_sum_0c1',
    'price_spread_sum_1c1',
    'price_spread_sum_3c1',
    'price_spread_sum_4c1',
    'price_spread_sum_6c1',   
    'bid_spread_sum_0c1',
    'bid_spread_sum_1c1',
    'bid_spread_sum_3c1',
    'bid_spread_sum_4c1',
    'bid_spread_sum_6c1',       
    'ask_spread_sum_0c1',
    'ask_spread_sum_1c1',
    'ask_spread_sum_3c1',
    'ask_spread_sum_4c1',
    'ask_spread_sum_6c1',   
    'volume_imbalance_sum_0c1',
    'volume_imbalance_sum_1c1',
    'volume_imbalance_sum_3c1',
    'volume_imbalance_sum_4c1',
    'volume_imbalance_sum_6c1',       
    'bid_ask_spread_sum_0c1',
    'bid_ask_spread_sum_1c1',
    'bid_ask_spread_sum_3c1',
    'bid_ask_spread_sum_4c1',
    'bid_ask_spread_sum_6c1',
    'size_tau2_0c1',
    'size_tau2_1c1',
    'size_tau2_3c1',
    'size_tau2_4c1',
    'size_tau2_6c1'
]

train = pd.merge(train, mat1[nnn], how='left', on='time_id')
test = pd.merge(test, mat2[nnn], how='left', on='time_id')
print(f"train: {train.shape} \ntest: {test.shape}")

del mat1,mat2
gc.collect()

## HistGradientBoostingRegressor

In [ ]:
def train_and_evaluate_gbr(train, test):
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    
    y = train['target']
    oof_predictions = np.zeros(train.shape[0])
    test_predictions = np.zeros(test.shape[0])
    
    kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        
        weights = 1 / np.square(y_train)
        
        model = HistGradientBoostingRegressor(
            #max_depth=8,
            learning_rate=0.09,
            max_iter=1500,
            max_leaf_nodes=112, 
            min_samples_leaf=500, 
            early_stopping=True,
            n_iter_no_change=100,
            random_state=2021
        )
        
        model.fit(x_train[features], y_train, sample_weight=weights)
        
        y_pred = model.predict(x_val[features])
        oof_predictions[val_ind] = y_pred
        test_predictions += model.predict(test[features]) / 5
        
        joblib.dump(model, f'./gbr_model_{fold + 1}C.txt')
        
        rmspe_score = rmspe(y_val, y_pred)
        print(f'Fold: {fold + 1} - OOF RMSPE: {rmspe_score}')
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'\nAll folds - OOF RMSPE: {rmspe_score}')
    
    return test_predictions

In [ ]:
predictions_gbr = train_and_evaluate_gbr(train, test)

## LGBM

In [ ]:
def train_and_evaluate_lgb(train, test, params):
    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    
    y = train['target']
    oof_predictions = np.zeros(train.shape[0])
    test_predictions = np.zeros(test.shape[0])
    
    kfold = KFold(n_splits = 5, random_state = 2021, shuffle = True)
    
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        
        model = lgb.train(params = params,
                          num_boost_round = 1300,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 200,
                          early_stopping_rounds = 50,
                          feval = feval_rmspe)
        
        y_pred = model.predict(x_val[features], num_iteration=model.best_iteration)
        oof_predictions[val_ind] = y_pred
        test_predictions += model.predict(test[features], num_iteration=model.best_iteration) / 5
        
        model.save_model(f'./lgb_model_{fold + 1}C.txt', num_iteration=model.best_iteration)
        
        rmspe_score = rmspe(y_val, y_pred)
        print(f'\nFold: {fold + 1} - OOF RMSPE: {rmspe_score}\n')
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'\nAll folds - OOF RMSPE: {rmspe_score}')
    
    return test_predictions

In [ ]:
seed = 2021

params = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    #'max_depth': -1,
    'max_leaves': 112,
    'max_bin': 100,
    'min_data_in_leaf': 500,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.35,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column': [0],
    'seed': seed,
    'feature_fraction_seed': seed,
    'bagging_seed': seed,
    'drop_seed': seed,
    'data_random_seed': seed,
    'n_jobs': -1,
    'verbose': -1
}

predictions_lgb = train_and_evaluate_lgb(train, test, params)

## Keras DNN

In [ ]:
# Generate kfolds based on the knn++ algorithm
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')
out_train = out_train.fillna(out_train.mean())

nfolds = 5
index = []
totDist = []
values = []

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = out_train.values
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds)

mat = np.c_[mat, np.arange(mat.shape[0])]
lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]

for n in tqdm(range(nfolds)):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

for n in tqdm(range(nfolds)):
    values.append([lineNumber[n]])    

s=[]
for n in tqdm(range(nfolds)):
    s.append(mat[lineNumber[n],:])    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in tqdm(range(nind-1)):    
    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
        
        f = totDist[cycle]/np.sum(totDist[cycle])
        j = 0
        kn = 0
        
        for val in f:
            j += val        
            if (j > luck[cycle]):
                break
            kn +=1
        
        lineNumber[cycle] = kn
        
        for n_iter in range(nfolds):    
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)

for n_mod in tqdm(range(nfolds)):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)

qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()

for col in tqdm(colNames):
    qt = QuantileTransformer(random_state=21,
                             n_quantiles=2000, 
                             output_distribution='normal')
    
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

In [ ]:
train_nn[['stock_id','time_id','target']] = train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']] = test[['stock_id','time_id']]
print(f"train_nn: {train_nn.shape} \ntest_nn: {test_nn.shape}")

In [ ]:
# Generate aggregate features
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()
ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    
mat = []
matTest = []

n = 0
for ind in l:
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'], inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2, mat1.loc[mat1.time_id==5]])

mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
nnn = [
    'time_id',
    'log_return1_realized_volatility_0c1',
    'log_return1_realized_volatility_1c1',     
    'log_return1_realized_volatility_3c1',
    'log_return1_realized_volatility_4c1',     
    'log_return1_realized_volatility_6c1',
    'total_volume_sum_0c1',
    'total_volume_sum_1c1', 
    'total_volume_sum_3c1',
    'total_volume_sum_4c1', 
    'total_volume_sum_6c1',
    'trade_size_sum_0c1',
    'trade_size_sum_1c1', 
    'trade_size_sum_3c1',
    'trade_size_sum_4c1', 
    'trade_size_sum_6c1',
    'trade_order_count_sum_0c1',
    'trade_order_count_sum_1c1',
    'trade_order_count_sum_3c1',
    'trade_order_count_sum_4c1',
    'trade_order_count_sum_6c1',      
    'price_spread_sum_0c1',
    'price_spread_sum_1c1',
    'price_spread_sum_3c1',
    'price_spread_sum_4c1',
    'price_spread_sum_6c1',   
    'bid_spread_sum_0c1',
    'bid_spread_sum_1c1',
    'bid_spread_sum_3c1',
    'bid_spread_sum_4c1',
    'bid_spread_sum_6c1',       
    'ask_spread_sum_0c1',
    'ask_spread_sum_1c1',
    'ask_spread_sum_3c1',
    'ask_spread_sum_4c1',
    'ask_spread_sum_6c1',   
    'volume_imbalance_sum_0c1',
    'volume_imbalance_sum_1c1',
    'volume_imbalance_sum_3c1',
    'volume_imbalance_sum_4c1',
    'volume_imbalance_sum_6c1',       
    'bid_ask_spread_sum_0c1',
    'bid_ask_spread_sum_1c1',
    'bid_ask_spread_sum_3c1',
    'bid_ask_spread_sum_4c1',
    'bid_ask_spread_sum_6c1',
    'size_tau2_0c1',
    'size_tau2_1c1',
    'size_tau2_3c1',
    'size_tau2_4c1',
    'size_tau2_6c1'
] 

train_nn = pd.merge(train_nn, mat1[nnn], how='left', on='time_id')
test_nn = pd.merge(test_nn, mat2[nnn], how='left', on='time_id')
print(f"train_nn: {train_nn.shape} \ntest_nn: {test_nn.shape}")

del mat1,mat2
del train,test
gc.collect()

In [ ]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))


def base_model(n_feats):
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(n_feats,), name='num_data')

    num_out = keras.layers.Dense(160, activation='swish')(num_input)
    
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, 
                                            stock_embedding_size, 
                                            input_length=1, 
                                            name='stock_embedding')(stock_id_input)
    
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_out])
    
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)

    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
        inputs = [stock_id_input, num_input],
        outputs = out,
    )
    
    return model

In [ ]:
get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128, 64, 32)
stock_embedding_size = 32

cat_data = train_nn['stock_id']

target_name='target'
pred_name = 'pred_NN'

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min', restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, 
    verbose=0, mode='min')

In [ ]:
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=2020)
counter = 1

features_to_consider = list(train_nn)
features_to_consider.remove('time_id')
features_to_consider.remove('target')

try:
    features_to_consider.remove('pred_NN')
except:
    pass

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])
oof_predictions_nn = pd.DataFrame(np.zeros((train_nn.shape[0], 1)), columns=['target'])

for n_count in range(n_folds):
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    num_data = X_train[features_to_consider]
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    
    model = base_model(num_data.shape[1])
    model.compile(
        keras.optimizers.Adam(learning_rate=0.005),
        loss=root_mean_squared_per_error
    )
    
    chkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'./dnn_model_{counter}C.h5', 
        monitor='val_loss', verbose=0, 
        save_best_only=True, mode='min')

    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=1000,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau, chkpoint],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose=0)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    oof_predictions_nn.loc[train_nn.time_id.isin(values[n_count]), target_name] = preds
    
    score = round(rmspe(y_true = y_test, y_pred = preds), 5)
    print(f'Fold: {counter} - OOF RMSPE: {score}')
    
    tt = scaler.transform(test_nn[features_to_consider].values)
    test_predictions_nn += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10) / n_folds
       
    counter += 1
    features_to_consider.append('stock_id')
    
rmspe_score = rmspe(train_nn['target'].values.ravel(), oof_predictions_nn.values.ravel())
print(f'\nAll folds - OOF RMSPE: {rmspe_score}')

## Create submission file

In [ ]:
test_nn["row_id"] = test_nn["stock_id"].astype(str) + "-" + test_nn["time_id"].astype(str) 
test_nn[target_name] = (test_predictions_nn * 0.55) + (predictions_lgb * 0.3) + (predictions_gbr * 0.15)

display(test_nn[['row_id', target_name]].head())
test_nn[['row_id', target_name]].to_csv('submission.csv', index = False)